In [2]:
import numpy as np
from Modeling import soeModel
%load_ext autoreload
%autoreload 2
import pandas as pd
import re
import json

In [124]:
def eq(s1, s2):
    s1_sorted = s1.sort_values().reset_index(drop=True)
    s2_sorted = s2.sort_values().reset_index(drop=True)
    diff = (s1_sorted - s2_sorted).abs().max()
    return diff < 1e-10

def compare_dicts(dict1, dict2, margin=1e-3):
    if dict1.keys() != dict2.keys():
        return False, "Keys are different"

    for key in dict1.keys():
        if isinstance(dict1[key], float) and isinstance(dict2[key], float):
            if not (abs(dict1[key] - dict2[key]) <= margin):
                return False, f"Difference found in key: {key}"
        elif dict1[key] != dict2[key]:
            return False, f"Difference found in key: {key}"

    return True, None



In [90]:
# new
model = soeModel()
model.preprocessing_2()
model.translate_cells('local_dictionary.json', False)
model.correct_digital_type()

In [4]:
# old
model_2 = soeModel()
model_2.preprocessing()
model_2.translate_cells('dict.json', False)
model_2.correct_digital_type()

In [119]:
params = {
    "types": "power",
    "pct": 0.5,
    "year": 2020,
    "coal_price": 343,
    "coal_power": True,
    "coal_purchase": False,
    "coal_transportation": False
}
table = model.scenario2(**params)
# table_2 = model_2.scenario2(**params)

/Users/zechengli/Desktop/CHINA_SOE_MODEL/Modeling.py:471: RuntimeWarning: invalid value encountered in divide
  'Power Dispatched (TWh)': power_dispatch,


In [141]:
model.case

,Reduced by 50%
Power Dispatched (TWh),61.635
Weighted Average Tariff (RMB/MWh),343
Estimated Gross Revenue (RMB/MWh),21140.805
Estimated Sales Cost (Million RMB),24226.659
Gross Profit (Million RMB),-3085.854
Gross Profitability,-14.60%


In [121]:
table_2 = model_2.scenario(**params)

/Users/zechengli/Desktop/CHINA_SOE_MODEL/Modeling.py:350: RuntimeWarning: invalid value encountered in divide
  lst2 = (np.array(lst1) / np.array(forecasted_gross_profit_lst_copy)).tolist()


In [11]:
model_2.case

,Reduced by 50%
Power Dispatched (TWh),61.635
Weighted Average Tariff (RMB/MWh),343
Estimated Gross Revenue (RMB/MWh),21140.805
Estimated Sales Cost (Million RMB),24226.659
Gross Profit (Million RMB),-3085.854
Gross Profitability,-12.74%


In [133]:
compare_dicts(model.data, model_2.data,margin=100)

(False, 'Difference found in key: Weighted Average Tariff (RMB/MWh)')

In [84]:
year = 2020
electricity_sale_price = \
            model.power_sale.loc[year]['Total electricity sales (100 million kWh)'] / \
            model.power_sale.loc[year]['Total electricity sales (100 million kWh)'].sum() * \
            model.power_sale.loc[year]['Electricity sales price (RMB/MWh)']

group = model.power_sale.groupby(['years', 'Power Plant Classification'])
group_filtered = group.filter(lambda x: x.name[0] == year)

coal_types_total = group.sum(numeric_only=True).loc[year]

power_dispatch = coal_types_total['Total electricity sales (100 million kWh)'].values.tolist()
total_sales = coal_types_total['Total electricity sales (100 million kWh)'].sum()
power_dispatch.insert(0, total_sales)

weighted_average_tariff = group_filtered.groupby('Power Plant Classification').apply(
    lambda df: (
        df['Total electricity sales (100 million kWh)']
        / df['Total electricity sales (100 million kWh)'].sum()
        * df['Electricity sales price (RMB/MWh)']
    ).sum()
).values.tolist()
weighted_average_tariff.insert(0, electricity_sale_price.sum())

In [87]:
weighted_average_tariff

[333.44057971014485, 326.9755820556501, 563.0, 232.0, 0.0]

In [64]:
year = str(2020)
electricity_sale_price = \
            model_2.power_sale['Total electricity sales (100 million kWh) ' + year] / \
            model_2.power_sale['Total electricity sales (100 million kWh) ' + year].sum() * \
            model_2.power_sale['Electricity Sales Price (RMB/MWh) ' + year]

group2 = model_2.power_sale.groupby('Power Plant Classification')[
            ['Total electricity sales (100 million kWh) ' + year, 'Electricity Sales Price (RMB/MWh) ' + year]]

weighted_average_tariff2 = group2.apply(lambda df: (df['Total electricity sales (100 million kWh) ' + year]
                                                          / df[
                                                              'Total electricity sales (100 million kWh) ' + year].sum()
                                                          * df[
                                                              'Electricity Sales Price (RMB/MWh) ' + year]).sum()
                                              ).values.tolist()
weighted_average_tariff2.insert(0, electricity_sale_price.sum())

In [18]:
weighted_average_tariff2

[333.4405797101449, 326.9755820556502, 563.0, 232.0, 0.0]

In [21]:
group

,Total power generation (100 million kWh),Total electricity sales (100 million kWh),Average utilization hours (hours),Standard coal consumption for electricity sales (g/kWh),Electricity sales price (RMB/MWh),Total installed capacity at the end of the year (MW),Equity installed capacity at the end of the year (MW)
Power Plant Classification,,,,,,,
coal burning,1318.5,1232.7,64571.0,4431.0,4710.0,31204.0,21658.0
gas,38.4,37.5,4045.0,192.0,563.0,950.0,950.0
hydropower,6.4,6.3,5124.0,0.0,232.0,125.0,48.0
wind energy,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [22]:
group2

In [ ]:
model.power_sale.set_index('years')['Electricity sales price (RMB/MWh)'][2019]

In [189]:
with open("local_dictionary.json", "r") as f:
    translations_all = json.load(f)

In [205]:
dict_en = {v: k for k, v in translations_all.items()}

In [263]:
s = 'Cost (RMB million)'.lower()
# dict_en

In [ ]:
translations_all['总发电量 (亿千瓦时)']

In [ ]:
# model.power_sale

In [ ]:
model.correct_digital_type()

In [ ]:
model.power_sale

In [ ]:
params = {
    "types": "power",
    "pct": 0.5,
    "year": 2019,
    "coal_price": 50.0,
    "coal_power": True,
    "coal_purchase": False,
    "coal_transportation": False
}
table = model.scenario(**params)

In [ ]:
table

In [ ]:
model.case

In [ ]:
#TODO: change the table structure

In [ ]:
model = soeModel()
is_cost = False
df = model.power_sale.copy()
######
# column names reformatting
df.loc[0] = df.loc[0].str.extract(r'(\d{4})', expand=False).fillna(method='ffill')
df = df.fillna('')
df.iloc[1] = df.iloc[1].str.replace('\n', ' ').str.strip()
columns = df.iloc[1] + ' ' + df.iloc[0]

# if the data is a cost-related table
if is_cost:
    columns = columns.replace(' ', '成本类别')
df.columns = columns

# drop the unnecessary rows
df = df.iloc[2:].reset_index(drop=True)
df.columns = [col.strip() for col in df.columns]

# Identify the columns containing a year in the first row
year_columns = [col for col in df.columns if re.search(r'\d{4}$', col)]
new_column_names = {col: re.sub(r'\s*\d{4}\s*$', '', col) for col in df.columns}
# Preprocessing translate to the time-series panel data
for col in year_columns:
    match = re.search(r'\d{4}', col)
    year = match.group(0)
    df[col] = df[col].apply(lambda x: str(x) + '!' + str(year))
    # If Cost Category
    if is_cost:
        df[col] = df['成本类别'] + '!' + df[col]
    # if power sale
    else:
        df[col] = df['电厂分类'] + '!' + df['电厂'] + '!' + df['所在电网'] + '!' + df['地理位置'] + '!' + df[col]

# Drop and will be adding back afterward
if is_cost:
    df.drop(['成本类别'], axis=1, inplace=True)
else:
    df.drop(['电厂分类','电厂','所在电网','地理位置'], axis=1, inplace=True)

# Rename the cok
df.rename(columns=new_column_names, inplace=True)

# Drop the unnecessary columns
if not is_cost:
    df.drop(['预测毛利润 (百万元)','计算I38中的加权平均售电电价','计算Q38中的加权平均售电电价'], axis=1, inplace=True)
# Get unique column names
unique_columns = df.columns.unique()

# Create an empty DataFrame to store the concatenated columns
concatenated_df = pd.DataFrame()

# Loop through unique column names and concatenate the columns vertically
for unique_col in unique_columns:
    # Create an empty list to store values
    values = []

    # Extend the values list with non-NaN values from the column
    values.extend(df[unique_col].values.tolist())

    # Flatten the nested list
    if len(values) != 1:
        values = [item for sublist in values for item in sublist]

    # Add the values list as a column in the concatenated DataFrame
    concatenated_df[unique_col] = pd.Series(values)


# Replace the original DataFrame with the concatenated one
df = concatenated_df

# Formal step of translating to the time-series panel data
for unique_col in unique_columns:
    if is_cost:
        df[['成本类别', unique_col, '年份']] = df[unique_col].str.split('!', expand=True)
    else:
        df[['电厂分类','电厂','所在电网','地理位置', unique_col, '年份']] = df[unique_col].str.split('!', expand=True)

# let cost category as the first column
if is_cost:
    columns_to_move = '成本类别'
    cols = df.columns.tolist()
    cols.insert(0, cols.pop(cols.index(columns_to_move)))
    df = df[cols].sort_values(by=['年份']).reset_index(drop=True)
# power sale
else:
    columns_to_move = ['电厂分类', '电厂', '所在电网', '地理位置']
    # Reorder columns by moving the specified columns to the front
    new_columns = columns_to_move + [col for col in df.columns if col not in columns_to_move]
    # Update the DataFrame with the new column order
    df = df[new_columns].sort_values(by=['年份']).reset_index(drop=True)

# Empty Values processing
df.replace(['-', '/', '  -   ', ' – ', ''], np.nan, inplace=True)
df.replace({r'\n': ''}, regex=True, inplace=True)

In [ ]:
df.columns

In [ ]:
s1 = df.set_index('年份')['总发电量 (亿千瓦时)']['2021']

In [ ]:
df

In [ ]:
s1

In [ ]:
s2 = model.power_sale[4][2:]

In [ ]:
s2

In [ ]:
# Sort the values in each series
s1_sorted = s1.sort_values().reset_index(drop=True)
s2_sorted = s2.sort_values().reset_index(drop=True)
# Get a boolean mask for where the sorted series are not equal

In [ ]:
print(s1_sorted.equals(s2_sorted))  # Output: True